In [27]:
import os
# print(os.getcwd())
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
import pickle
import sklearn
print(sklearn.__version__)
import pandas as pd
import numpy as np
import sys
from evaluation.metric import computeAccuracy, computeF1Score, computeAUC, computeAUCBehavior
# from sklearn.metrics import roc_auc_score

# %load_ext autoreload
# %autoreload
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from scipy.stats import uniform
# %reload_ext pipelines.alcohol.AlcoholPipeline
from pipelines.alcohol import AlcoholPipeline
from sklearn.grid_search import RandomizedSearchCV, GridSearchCV
from scripts.gridsearch import text_grid
from sklearn.externals import joblib


0.17


In [28]:
# clf_alc_initial = pickle.load(open('./alcohol classifiers/clf_alc_UPDATED.p', 'rb'))
# clf_fpa_initial = pickle.load(open('./alcohol classifiers/clf_fpa_UPDATED.p', 'rb'))
clf_behavior_initial = pickle.load(open('./alcohol classifiers/clf_fpl_double_labeled', 'rb'))
df = pd.read_csv("./data/alcohol_training_instances.csv")

Prepare the behavior level data

In [29]:
# appending labels based on whether tweet label contains phrase
behavior_labels = []
behavior_index_in_df = []

fpl = "'first_person_level'"

# extracting FP levels and constructing subset vector simultaneously
for i in range(len(df)):
    if fpl in df.labels[i]:
        behavior_labels.append(int(df.labels[i][df.labels[i].find(fpl) + len(fpl) + 2]))
        behavior_index_in_df.append(1)
    else:
        behavior_index_in_df.append(0)

behavior_labels = list(map(int, behavior_labels))
print(behavior_labels[:20])
# print(subset_vec_alc_fp[:10])
print('length of first person level labels: ', len(behavior_labels)) # 6357
print(behavior_index_in_df[:20])
print('length of first person level index in df: ', len(behavior_index_in_df)) # 15651
index_behavior = list(np.where(behavior_index_in_df)[0])
print(index_behavior[:15])
print('length of first person level df index: ', len(index_behavior)) # 6357
df_behavior = df.iloc[index_behavior,]
# df_fp.head
# fp_labels = np.asarray(fp_labels)

# change label 3 to label 0
behavior_labels = [0 if x == 3 else x for x in behavior_labels] 

# train test dataset split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_behavior, behavior_labels, test_size=0.2, random_state=26)

[0, 0, 2, 2, 1, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0]
length of first person level labels:  6357
[0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0]
length of first person level index in df:  15651
[2, 5, 6, 7, 11, 13, 17, 18, 21, 22, 23, 24, 26, 27, 28]
length of first person level df index:  6357


In [30]:
print(len(behavior_labels))
print(len(behavior_index_in_df))
# behavior_labels[:30]
# for i in behavior_labels:
#     if i ==3:
#         print(True)

6357
15651


Create a model for the following grid search

In [31]:
# SVM
clf_behavior_SVM = AlcoholPipeline(global_features=["text"]).pipeline(LinearSVC())
clf_behavior_SVM.steps

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
      ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=1, loss='squared_hinge', max_iter=1000,
       multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
       verbose=0))]

In [32]:
clf_behavior_SVM.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

Now we are trying to implement grid search.

In [33]:
param_grid = {
    'clf__C': uniform(0.0001, 1000),
    'clf__penalty': ['l2', "l1"],
#     'clf__penalty': ['l2'],
    'clf__tol': uniform(0.0001, 0.001),
#     'clf__verbose': [0]
#     'features__text__tfidf__analyzer': ['char'],
#     'features__text__tfidf__ngram_range': [(2,5)]
}

param_grid.update(text_grid)

cv_kwargs = dict(
    n_iter=50,
    scoring=None,
    fit_params=None,
    n_jobs=-1,
    iid=True,
    refit=True,
    cv=10,
    verbose=0,
    pre_dispatch='2*n_jobs',
    error_score=0
)

In [34]:
clf_behavior_SVM_grid = RandomizedSearchCV(clf_behavior_SVM, param_grid, **cv_kwargs)
clf_behavior_SVM_grid.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


RandomizedSearchCV(cv=10, error_score=0,
          estimator=Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
          fit_params={}, iid=True, n_iter=50, n_jobs=-1,
          param_distributions={'clf__C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000028E0540A320>, 'clf__penalty': ['l2', 'l1'], 'clf__tol': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000028E05429128>, 'features__text__tfidf__analyzer': ['char', 'word'], 'features__text__tfidf..., 'english'], 'features__text__tfidf__tokenizer': [None, <function tokenize at 0x0000028E03236378>]},
          pre_dispatch='2*n_jobs', random_state=None

In [35]:
# get the best estimator 
best_estimator_SVM = clf_behavior_SVM_grid.best_estimator_
print('best estimator: ', best_estimator_SVM)
print('-------------------------- mean accuracy on the cross validation of best estimator --------------------------')
print('best score: ', clf_behavior_SVM_grid.best_score_)
print('best score related parameters: ', clf_behavior_SVM_grid.best_params_)

best estimator:  Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=F...00, multi_class='ovr', penalty='l2', random_state=None,
     tol=0.0010541708034333423, verbose=0))])
-------------------------- mean accuracy on the cross validation of best estimator --------------------------
best score:  0.640511307767945
best score related parameters:  {'clf__C': 251.59075109196576, 'clf__penalty': 'l2', 'clf__tol': 0.0010541708034333423, 'features__text__tfidf__analyzer': 'word', 'features__text__tfidf__lowercase': False, 'features__text__tfidf__max_features': 99765, 'features__text__tfidf__min_df': 1, 'features__text__tfidf__ngram_range': (1, 3), 'features__text__tfidf__norm': 'l1', 'features__text__tfidf__stop_words': None, 'features__tex

Test the best estimator performance on testing dataset

In [36]:
# predict in testing set.
y_behavior_SVM_grid = best_estimator_SVM.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_behavior_SVM_grid, y_test)
f1_score_SVM_grid = computeF1Score(y_test, y_behavior_SVM_grid, average = None)
print('F1 Score: ', f1_score_SVM_grid)
auc_score_SVM_grid = computeAUCBehavior(best_estimator_SVM, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('SVM classifier AUC: ', auc_score_SVM_grid)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  1272
492 different item in whole dataset
accuracy: 61.320755%
F1 Score:  [0.70159453 0.57631954 0.44106464]
SVM classifier AUC:  {'current': 0.7472408368532535, 'looking': 0.7876124744533872, 'reflecting': 0.7478516622062713, 'micro': 0.7824014328942683, 'macro': 0.7615474208016202}


Save the best estimator

In [37]:
joblib.dump(best_estimator_SVM, './alcohol classifiers/behaviorBestEstimatorSVM.pkl', compress=1)

['./alcohol classifiers/behaviorBestEstimatorSVM.pkl']

Plot the grid search result in table. 

In [38]:
cv = pd.DataFrame(clf_behavior_SVM_grid.grid_scores_)
cv.cv_validation_scores[0]
cv

,parameters,mean_validation_score,cv_validation_scores
0,"{'clf__C': 770.6447695224987, 'clf__penalty': ...",0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"{'clf__C': 977.1116173791768, 'clf__penalty': ...",0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"{'clf__C': 964.2952641726399, 'clf__penalty': ...",0.545919,"[0.5588235294117647, 0.5549019607843138, 0.530..."
3,"{'clf__C': 489.8651724072061, 'clf__penalty': ...",0.547689,"[0.5588235294117647, 0.5529411764705883, 0.532..."
4,"{'clf__C': 387.4303002020613, 'clf__penalty': ...",0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5,"{'clf__C': 337.11018641210205, 'clf__penalty':...",0.542183,"[0.5470588235294118, 0.5588235294117647, 0.546..."
6,"{'clf__C': 181.2533686130598, 'clf__penalty': ...",0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
7,"{'clf__C': 928.0617505710735, 'clf__penalty': ...",0.573058,"[0.6058823529411764, 0.5588235294117647, 0.573..."
8,"{'clf__C': 543.6319027796051, 'clf__penalty': ...",0.557522,"[0.5686274509803921, 0.5764705882352941, 0.544..."
9,"{'clf__C': 309.71409162304, 'clf__penalty': 'l...",0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


Test the one you are saved

In [39]:
clf_saved = joblib.load('./alcohol classifiers/behaviorBestEstimatorSVM.pkl')

In [40]:
# predict in testing set.
y_behavior_SVM_saved = clf_saved.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_behavior_SVM_saved, y_test)
f1_score_SVM_saved = computeF1Score(y_test, y_behavior_SVM_saved, average = None)
print('F1 Score: ', f1_score_SVM_saved)
auc_score_SVM_saved = computeAUCBehavior(clf_saved, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('LR classifier AUC: ', auc_score_SVM_saved)

length of total comps:  1272
492 different item in whole dataset
accuracy: 61.320755%
F1 Score:  [0.70159453 0.57631954 0.44106464]
LR classifier AUC:  {'current': 0.7472408368532535, 'looking': 0.7876124744533872, 'reflecting': 0.7478516622062713, 'micro': 0.7824014328942683, 'macro': 0.7615474208016202}


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Saved model works well